# Install and load necesary packages

In [1]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Please don't change this cell
#df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596

# Split dataset
## Random Train and Test Split

In [3]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678


# Utils

In [4]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

# Your Solution

In [5]:
# Write your code here
# You are required to implement the required solution here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

MAE = 0 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.
RMSE = 0 # 0 is an intial value, you need to update this with the actual perofrmance of your implementation.



# Normalizing the training data by subtracting the user mean
# Calculating the mean rating for each user and reshape it to a column vector
user_mean = np.mean(train_ds, axis=1).reshape(-1, 1)

# Subtracting the user mean from each users ratings to normalize the data
train_ds_normalized = train_ds - user_mean

# Confirming that zeros in the original data remain zeros after normalization
train_ds_normalized[train_ds == 0] = 0

# Defining the maximum and minimum rating values
V_max = 5
V_min = 1

# Calculating the maximum possible distance between two user rating
dist_max = np.sqrt(n_items * (V_max - V_min) ** 2)

# Initializing a matrix to store similarities between users
similarities = np.zeros((n_users, n_users))

# Iterating over all pairs of users to compute similarities
for i in range(n_users):
    for j in range(n_users):
        if i != j:  # here it is checked that similarity is not computed with same user  
            # Creating a mask to identify items rated by both users
            mask = (train_ds_normalized[i] > 0) & (train_ds_normalized[j] > 0)
            if np.any(mask):  
                # Computing the Euclidean distance between the two users for common rated
                dist = np.sqrt(np.sum((train_ds_normalized[i, mask] - train_ds_normalized[j, mask]) ** 2))
                # Converting the distance to similarity score using AED formula
                similarities[i, j] = 1 - (dist / dist_max)
            else:
                # setting it to zero if no common items found
                similarities[i, j] = 0
        else:
            similarities[i, j] = 0


In [6]:
# Predicting ratings 
def predict_ratings(train_ds, test_ds, similarities, k):
    # Initializing the predicted ratings matrix with zeros
    predicted_ds = np.zeros((n_users, n_items))
    
    # Iterating over each user
    for u in range(n_users):
        # Iterating over each item
        for i in range(n_items):
            if test_ds[u, i] > 0:
                # Finding users who have rated item i
                similar_users = np.where(train_ds[:, i] > 0)[0]
                if len(similar_users) > 0:
                    # Getting top-k most similar users who have rated the item
                    top_k_users = similar_users[np.argsort(-similarities[u, similar_users])][:k]
                    if len(top_k_users) > 0:
                        # Predicting the rating as the mean rating of the top-k similar users
                        predicted_ds[u, i] = np.mean(train_ds[top_k_users, i])
                    else:
                        # using mean rating, if no top-k users found 
                        predicted_ds[u, i] = user_mean[u]
                else:
                    # using mean rating , if no similar user rated the item 
                    predicted_ds[u, i] = user_mean[u]
    
    return predicted_ds


In [7]:
k = 100

# Predicting and evaluating values
# Using the predict_ratings function to generate predictions for the test dataset
predicted_ds = predict_ratings(train_ds, test_ds, similarities, k)

# Evaluating the predictions using the evaluate function
MAE, RMSE = evaluate(test_ds, predicted_ds)

In [8]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.7862202291639753, RMSE: 0.9916976991295829
